In [58]:
#!/usr/bin/env python
# coding: utf-8
# credits: https://github.com/adil2604
# # Создание слоя в GeoJSON формате

# In[1]:
import asyncio
import time

import aiohttp
import pandas as pd
import nest_asyncio
nest_asyncio.apply()
import geopandas as gpd
import requests
from shapely.geometry import Point, LineString
from geopy.distance import geodesic
from bs4 import BeautifulSoup
import json
import geocoder

from geocoder import Batch



async def download_site(session, url, query,city):
    """Get data for single address."""
    params = {
        'apiKey': 'HcNvySMr8-wOc-9NNzHqq_ikPB82Cvt7ewftxQyjlwA',
        'searchText': query,
        'city': city
    }
    if 'с. 'not in query and 'пос. ' not in query:
        params['searchText'] = 'г. '+city+' '+query;
        
    async with session.get(url, params=params) as res:
        try:
            data = await res.json()
            if data['Response']['View'][0]['Result'][0]['MatchLevel']=='houseNumber':
                data = data['Response']['View'][0]['Result'][0]['Location']
                return [data['Address']['Label'], data['DisplayPosition']['Longitude'], data['DisplayPosition']['Latitude'],query]
            else:
                return None
        except:
            print("Error with {}".format(params['searchText']))


async def download_all_sites(geocode_df):
    """Helper function. Run all tasks."""
    async with aiohttp.ClientSession() as session:
        tasks = []
        url = 'https://geocoder.ls.hereapi.com/6.2/geocode.json'
        for index, query in geocode_df.iterrows():
            task = asyncio.ensure_future(download_site(session, url, query['searchText'],query['city']))
            tasks.append(task)

        return await asyncio.gather(*tasks, return_exceptions=True)


def geocode_data(name,path_csv_file):
    """Main driver function."""
    df = pd.read_csv(path_csv_file, sep=",", encoding="utf-8")
    geocode_df = pd.DataFrame(data={'recId': df.index, 'searchText': df['address1'], 'city':df['city']})
    columns = [
        'address_geocoded',
        'displayLongitude',
        'displayLatitude',
        'address'
    ]
    geocoded_df = pd.DataFrame(columns=columns)
    start_time = time.time()
    all_data = asyncio.get_event_loop().run_until_complete(download_all_sites(geocode_df))
    duration = time.time() - start_time
    print(f"Geocoded in {duration} sec.")
    invalid = 0
    for d in all_data:
        # print(d)

        if d is not None:
            geocoded_df = geocoded_df.append(pd.Series(d, index=geocoded_df.columns), ignore_index=True)
        else:
            invalid = +1
    print(geocoded_df.head())
    print('{}_geocoded_df.csv'.format(name))
    geocoded_df.to_csv('{}_geocoded_df.csv'.format(name))
    duration = time.time() - start_time
    print(f"Done in  {duration} seconds. {invalid} invalid address.")


In [42]:
geocode_data('terminals','terminals_all.csv')



Error with пос. Косшы, ул. Республики, д. 1
Error with с. Косшы, ЖМ Лесная Поляна, д. 12
Error with с. Косшы, ЖМ Лесная Поляна, д. 16
Error with с. Косшы, ЖМ Лесная поляна, д. 22
Error with с. Косшы, ЖМ Лесная Поляна, д. 2, ВП-16
Error with с. Косшы, ЖМ Лесная поляна, д. 17, ВП-5
Error with с. Косшы, ул. Республики, д. 3
Error with с. Косшы, ул. Республики, д. 18 Б
Error with с. Косшы, ЖМ Лесная поляна, д. 6
Error with с. Косшы, ул. Республики, д. 26
Error with г. Нур-Султан ул. А 98, д. 8
Error with г. Алматы 12 мкр, д. 9/2
Error with г. Алматы 1 мкр-н, д. 23
Error with г. Алматы 1 мкр-н, д. 68/4
Error with г. Алматы 2 мкр-н, д. 28А
Error with г. Алматы 4 мкр-н, д. 10 А
Error with г. Алматы 4 мкр-н, д. 27
Error with г. Алматы 4 мкр-н, д. 2 Б
Error with г. Алматы 4 мкр-н, д. 10г/1
Error with г. Алматы 5 мкр-н, д. 19Б
Error with г. Алматы 8 мкр-н, д. 41/1
Error with г. Алматы 8 мкр-н, д. 39
Error with г. Алматы 8 мкр-н, д. 37/1
Error with г. Алматы 8 мкр-н, д. 41/6
Error with г. Алматы 

Error with пос. Томарлы, ул. Колхозная, д. 22Б
Error with с. Томарлы, мкр. Жулдыз, ул. 13, стр. 11
Error with г. Атырау ул. Владимирского, б/н
Error with пос. Рембаза, пер. Гарышкер, д. 27А
Error with г. Есик ул. Абая, д. 117А
Error with г. Есик ул. Алматинская, д. 157
Error with г. Есик ул. Алматинская, д. 157
Error with г. Есик ул. Алтын Адам, д. 23
Error with с. Кайназар, ул. Алматинская, д. 72А
Error with г. Есик ул. Алтын адам, б/н
Error with с. Байтерек, ул. Алмерек Абыз, д. 31
Error with г. Есик 1 мкр-н, д. 5
Error with г. Есик мкр. Ынтымак, д. 43
Error with г. Есик 2 мкр-н, д. 14
Error with г. Есик ул.  Алтын Адам, д. 85 Б
Error with г. Есик ул. Валиханова, д. 24Б
Error with г. Есик ул. Кешубаева, д. 41
Error with г. Жанаозен ул. Абая, д. 5Б
Error with г. Жанаозен ул. Абая, д. 5Б
Error with г. Жанаозен ул. Абая, д. 5Б
Error with г. Жезказган ул. Абая, д. 63, кв. 3
Error with с. Кенгир, ул. Ауэзова, д. 27
Error with г. Жезказган ул. Улытауская, д. 4А
Error with г. Жетысай ул. Ша

Error with г. Павлодар ул. Якова Геринга, д. 1/1
Error with г. Петропавловск ул. Абая, д. 77
Error with г. Павлодар ул. Академика Бектурова, д. 16
Error with г. Павлодар ул. Астана, д. 7/1
Error with г. Петропавловск ул. Абая, д. 67
Error with с. Бесколь, ул. Институтская, д. 2Б
Error with г. Петропавловск ул. Абая, д. 96
Error with с. Бесколь, ул. Гагарина, д. 2
Error with г. Петропавловск Мамлютская трасса, б/н
Error with с. Бесколь, ул. Институтская, д. 2Б
Error with г. Петропавловск ул. Абая, д. 33
Error with с. Покровка, Мамлютская трасса, б/н
Error with г. Петропавловск ул. Абая, д. 67
Error with г. Петропавловск ул. Батыр Баяна, д. 58
Error with г. Петропавловск ул. Ахременко, д. 35
Error with г. Петропавловск ул. Букетова, д. 18
Error with г. Петропавловск ул. Букетова, д. 61
Error with г. Петропавловск ул. Валиханова, д. 30
Error with г. Петропавловск ул. Валиханова, д. 56
Error with г. Мамлютка ул. Гуденко, д. 3
Error with г. Петропавловск ул. Дусухамбетова, д. 17В
Error with

Error with г. Мерке ул. Исмаилова, д. 214 А
Error with г. Тараз ул. Кольцевая, д. 38
Error with г. Тараз ул. Мамбет Батыра, д. 2/8
Error with г. Тараз ул. Ниеткалиева, д. 7
Error with г. Тараз ул. Мамбет Батыра, д. 2/6
Error with г. Тараз ул. Кыпшакпаева, д. 92
Error with г. Кулан ул. Рахманберди, д. 2
Error with г. Тараз ул. Ниеткалиева, д. 70 А
Error with г. Мерке ул. Сарымолдаева, д. 146a
Error with г. Тараз ул. Сулейманова, д. 29
Error with г. Тараз ул. Тауке хана, д. 24
Error with г. Тараз ул. Ташкентская, д. 313
Error with г. Мерке ул. Турара Рыскулова, д. 119, кв. (офис) А
Error with г. Тараз ул. Турара Рыскулова, д. 5 У
Error with г. Тараз ул. Момышулы, д. 27
Error with г. Тараз ул. Ташкентская, д. 244
Error with г. Темиртау 8 мкр-н, д. 31/1
Error with г. Темиртау ул. 4, д. 51/3
Error with г. Туркестан 1 мкр-н, д. 13Б
Error with г. Кентау пр. Абая, д. 15/4
Error with г. Кентау пр. Яссауи, д. 105/2
Error with с. Икан, ул. Кунаева, уч-к 1834
Error with г. Уральск 4 мкр-н, д. 2/1


NameError: name 'geocoded_df' is not defined

In [50]:
df = pd.read_csv("Kaspi_terminals_geocoded_df.csv",sep=",",encoding='utf-8')

df.head()
gdf = gpd.GeoDataFrame(df,geometry = gpd.points_from_xy(df.displayLongitude,df.displayLatitude))
gdf.head()

,Unnamed: 0,address_geocoded,displayLongitude,displayLatitude,address,geometry
0,0,"бульвар Академика К. Скрябина 6, 010000 Нұр-Сұ...",71.41651,51.180260,"бульвар Академика Скрябина, д. 6",POINT (71.41651 51.18026)
1,1,"проспект Кабанбай батыра 45, 010000 Нұр-Сұлтан...",71.40968,51.109625,"ЖК Highvill, пр. Кабанбай батыра, д. 43 В",POINT (71.40968 51.10963)
2,2,"проспект Бауыржана Момышулы 2/7, 010010 Нұр-Сұ...",71.46321,51.134750,"ЖК Акбулак Таун, пр. Момышулы, д. 2/7",POINT (71.46321 51.13475)
3,3,"проспект Улы Дала 11, 010000 Нұр-Сұлтан, Қазақ...",71.42422,51.096420,"ЖК Арман Кала, пр. Улы Дала, д. 11/1",POINT (71.42422 51.09642)
4,4,"проспект Улы Дала 27, 010018 Нұр-Сұлтан, Қазақ...",71.44400,51.093670,"ЖК Арнау-1, пр. Улы Дала, д. 27",POINT (71.44400 51.09367)


In [51]:
gdf.to_file('terminals_geocoded.geojson',driver='GeoJSON')

In [59]:
geocode_data('branches','branches_all.csv')



Error with г. Отеген батыр ул. Батталханова, д. 5
Error with г. Боралдай ул. Бостанова, д. 1/3
Error with г. Алматы 8 мкр-н, д. 41/6
Error with г. Кульсары пр. Махамбета, д. 18а
Error with г. Петропавловск ул. Абая, д. 67, оф. 78
Error with г. Павлодар ул. Камзина, д. 166/1
Error with г. Павлодар ул. К.Сураганова, д. 2/2, кв. 1
Error with г. Тараз пр. Жамбыла, д. 123
Error with г. Тараз ЖК Атшабар, д. 1
Error with г. Тараз мкр. Мынбулак, д. 45, кв. 41
Error with г. Уральск мкр. Женис, д. 21, помещение 10, Лит.Б1
Error with г. Уральск ул. Курмангазы, д. 80/1
Error with г. Шымкент мкр. Катын копир, ул. Жанакурылыс, д. 4А
Error with г. Актобе ул. 101 Стрелковой Бригады, д. 8
Error with г. Аксай 4 мкр-н, д. 2А
Error with г. Актобе ул. Шернияза, д. 58
Error with г. Каскелен пр. Абылай хана, д. 66А
Error with г. Актобе 11 мкр-н, д. 112 Г, помещение 108
Geocoded in 5.0425567626953125 sec.
                                    address_geocoded  displayLongitude  \
0  улица Каныша Сатпаева 10, 01

In [60]:
df = pd.read_csv("branches_geocoded_df.csv",sep=",",encoding='utf-8')

gdf = gpd.GeoDataFrame(df,geometry = gpd.points_from_xy(df.displayLongitude,df.displayLatitude))
gdf.head()
gdf.to_file('branches_geocoded.geojson',driver='GeoJSON')

In [61]:
geocode_data('atms','atms_all.csv')

Error with г. Нур-Султан Нур-Султан, Сарыарка, 31А (Аян)
Error with г. Степногорск 7 мкр-н, д. 141/1
Error with пос. Косшы, ул. Республики, д. 3В
Error with г. Степногорск 6 мкр-н, д. 2
Error with г. Степногорск 6 мкр-н, д. 2
Error with г. Степногорск 1 мкр-н, д. 7
Error with пос. Коргалжын, ул. Болганбаева, д. 14
Error with г. Нур-Султан пр. Косшыгулулы, д. 9/1
Error with г. Ерейментау ул. А.Кунанбаева, д. 117А
Error with с. Косшы, ЖМ Лесная поляна, д. 20
Error with г. Нур-Султан ул. А-82, д. 8, ж/д вокзал "Астана Нұрлы жол"
Error with с. Красноярка
Error with с. Жалтыр, ул. Ленина, д. 2
Error with с. Косшы, ул. Республики, д. 3 (
Error with г. Нур-Султан ул. 20-40, д. 2
Error with г. Нур-Султан ул. Жансугирулы, д. 8/1
Error with г. Ерейментау ул. Касымова, д. 97
Error with г. Нур-Султан ул. Караменде Би Шакаулы, д. 12А
Error with г. Нур-Султан ул. Косшыгулулы, д. 9/1, ТЦ Шелковый путь
Error with г. Степногорск 9 мкр-н, д. 83
Error with г. Нур-Султан ул. 20-40, д. 2
Error with г. Нур-

Error with пос. Каратобе, ул. Курмангалиева, д. 23
Error with г. Аксай 10 мкр-н, д. 2В
Error with г. Аксай 10 мкр-н, д. 2Б
Error with г. Аксай 5 мкр-н, д. 21
Error with г. Аксай 4 мкр-н, д. 50
Error with г. Аксай 5 мкр-н, д. 21
Error with г. Аксай 10 мкр-н, д. 2Б, ТД  Жибек жолы
Error with г. Аксай 4 мкр-н, д. 50
Error with г. Аксай 4 мкр-н, ул. Дружбы Народов, д. 50
Error with г. Аксай 4 мкр-н, д. 8
Error with г. Аксай 4 мкр-н, ул. Дружбы Народов, д. 50
Error with г. Аксай пр. Дружбы Народов, д. 1В
Error with пос. Жымпиты, ул. Казахстанская, д. 2А
Error with г. Аксай 3 мкр-н, д. 1А
Error with г. Аксай кв. Чешский Городок
Error with г. Аксай ул. Дружбы Народов, д. 7 В
Error with г. Аксай ул. Промзона, д. 99 Н
Error with г. Аксай ул. Дружбы Народов, д. 1/8
Error with пос. Каратобе, ул. С.Датулы, д. 17
Error with г. Аксай 4 мкр-н, д, 50
Error with г. Аксай ул. Дружбы Народов, д. 1/8 В
Error with г. Аксай 5 мкр-н, д. 8А
Error with г. Аксай ул. С.Датова, д. 4
Error with г. Аксай ул. Дружбы

Error with г. Кандыагаш ул. Привокзальная, д. 1
Error with г. Актобе ул. Тилеу Батыра, д. 10
Error with г. Актобе ул. Таракулы, д. 25А
Error with г. Актобе ул. Ш.Калдаякова, д. 33А
Error with г. Актобе ул. Ш.Калдаякова, д. 33А
Error with г. Актобе ул. Ш.Калдаякова, д. 24-4
Error with г. Атырау мкр. Авангард, рынок Коктем
Error with пос. Доссор, мкр. Нефтянник
Error with пос. Индерборский, ул. Кунаева, д. 14
Error with г. Атырау мкр. Балыкшы, ул. Абая Кунанбаева, д. 17В
Error with г. Атырау пересечение пр. Азаттык и ул. Ауэзова
Error with пос. Индерборский, ул. Мендигалиева, д. 30
Error with г. Атырау мкр. Балыкшы, ул. Кунанбаева, д. 28а
Error with пос. Балыкшы, ул. Кунанбаева, д. 28А
Error with г. Атырау мкр. Балыкшы, ул. А.Кунанбаева, д. 28А
Error with г. Атырау мкр. Сары-Арка, ул. Кулманова, д. 40
Error with пос. Тенгиз, 5 мкр-н, д. 17
Error with пос. Тенгиз, 3 мкр-н, д. 4/5
Error with г. Атырау пр. И.Тайманова, д. 1/2
Error with г. Атырау пр. И.Тайманова, д. 1/2
Error with г. Атырау

Error with г. Кокшетау ул. М.Горького, д. 4/2
Error with г. Кокшетау ул. Ашимова, д. 2А
Error with г. Кокшетау Восточная промзона, проезд 20, уч-к 18 А
Error with г. Атбасар ул. Ауэзова, д. 14А
Error with г. Державинск ул.  Ленина, д. 28
Error with г. Державинск ул.  Ленина, д. 28
Error with с. Отар, ПГТ Гвардейский
Error with г. Кордай ул. Казыбеке Би, д. 18
Error with с. Толе би, ул. Толе би, д. 264
Error with с. Сортобе, ул. Ленина, д. 182А
Error with г. Шу ул. Науалиева, д. 5А
Error with г. Шу ул. Сатпаева, д. 42
Error with г. Кордай ул. Толе би, д. 100
Error with г. Шу ул. Науалиева, д. 5А
Error with г. Житикара 11 мкр-н, д. 30
Error with г. Житикара 2 мкр-н, д. 31
Error with г. Житикара 2 мкр-н, д. 31
Error with г. Житикара 2 мкр-н, д. 31
Error with г. Житикара 2 мкр-н, д. 31
Error with г. Житикара 11 мкр-н, д. 26
Error with г. Житикара 6 мкр-н, д. 55
Error with г. Житикара 6 мкр-н, д. 53
Error with пос. Затобольск, ул. Калинина, д. 51/1
Error with пос. Сарыколь, ул. Тауелыздык, 

Error with г. Петропавловск ул. Ибраева, д. 22А
Error with г. Петропавловск ул. К.Сутюшева, д. 58Б
Error with г. Петропавловск ул.Кошукова, 2
Error with г. Петропавловск ул. Конституции Казахстана, д. 24
Error with г. Петропавловск ул. К.Сутюшева, д. 21
Error with г. Петропавловск ул. К.Сутюшева, д. 21
Error with г. Мамлютка ул. Ленина, д. 51
Error with г. Петропавловск ул. К.Сутюшева, д. 58 Б
Error with г. Петропавловск ул. Кошукова, 1, магазин "Семейный"
Error with г. Петропавловск ул. Кошукова, д. 2
Error with г. Петропавловск ул. Н.Назарбаева, д. 242Б
Error with г. Петропавловск ул. Н.Назарбаева, д. 242Б
Error with г. Петропавловск ул. Мира, д. 229
Error with г. Петропавловск ул. Сатпаева, д. 38
Error with г. Петропавловск ул. Хименко, д. 3
Error with г. Петропавловск ул. Назарбаева, д. 252
Error with г. Петропавловск ул. Сатпаева, д. 27
Error with г. Петропавловск ул. Абая, д. 77
Error with г. Петропавловск ул. Абая, д. 77
Error with г. Петропавловск ул. Муканова, д. 53, гипермарк

Error with г. Тараз пр. Жамбыла, д. 123
Error with г. Тараз пр. Жамбыла, д. 200
Error with г. Каратау мкр. Шакирова, д. 18
Error with г. Тараз мкр. Телецентр, д. 11
Error with с. Бурыл, ул. Конаева, д. 1Г
Error with г. Тараз мкр. Талас, д. 2
Error with пос. Мынарал, Цементный завод
Error with г. Тараз мкр. Телецентр, д. 2
Error with г. Тараз пр. Жамбыла, д. 10
Error with г. Тараз ул. Абая, д. 157 А
Error with с. Аса, ул. Абая, д. 119
Error with г. Тараз с.Аса, ул. Абая, д. 64
Error with с. им. Бауржан Момышулы, ул. Рысбек Батыра, б/н
Error with г. Тараз пр. Жамбыла, д. 18А
Error with с. Аса, ул. Абая, д. 64
Error with с. Б.Момышулы, ул. Коктем, д. 17
Error with г. Тараз Тараз, Ташкентская, 118
Error with г. Тараз ул. Абая, д. 98
Error with г. Тараз ул. Абая, д. 142
Error with с. Аса, ул. Абая, д. 36А
Error with г. Тараз ул. Абая, д. 120В
Error with с. Масанчи, ул. Советская, б/н
Error with с. Б.Момышулы, ул.Султан Бейбарыс, 1
Error with с. Шолаккорган, ул. Жибек жолы, д. 71
Error with 

Error with г. Шымкент мкр. Асар, ул. Жана курылыс, д. 1945
Error with пос. Коктерек, б/н
Error with пос. Атакент, ул. Н.Торекулова, д. 1
Error with с. Аксукент, ул. Джандарбекова б/н
Error with с. Ленгер, ул. Новостроевка, б/н
Error with г. имени Турара Рыскулова село им. Бауыржан Момышулы, ул. Рысбек батыра, д. 100
Error with с. Дербисек, ул. Шонгара, д. 25А
Error with с. Ошакты, 116 квартал, уч.215
Error with с. Шаульдер, ул. Сарсенбаева, д. 13
Error with с. Темирлан, Кажымукана, б/н
Error with с. Шаульдир, ул. Алтынбекова, д. 19
Error with с. Тогыс, ул. Нефтеразветка, б/н
Error with с. Ынтымак, ул. Корикты, б/н
Error with с. Тюлькубас, ул. Байсерикова, д. 52А
Error with г. Ленгер ул. К.Рыскулбекова, б/н
Error with г. Ленгер ул. Толе би, д. 209
Error with г. Ленгер ул. Толе би, д. 58
Error with пос. Заозерное
Error with г. Щучинск санаторий "Алмаз"
Error with г. Ленгер ул. Толе би, д. 211
Error with г. Щучинск станция "Курорт Боровое"
Error with пос. Щучинский, ул. Приозерная, д. 17


In [62]:
df = pd.read_csv("atms_geocoded_df.csv",sep=",",encoding='utf-8')

gdf = gpd.GeoDataFrame(df,geometry = gpd.points_from_xy(df.displayLongitude,df.displayLatitude))
gdf.head()
gdf.to_file('atms_geocoded.geojson',driver='GeoJSON')

In [63]:
geocode_data('autoservices','autoservices_all.csv')

Error with г. Алматы ул. Ислама Каримова, д. 93
Error with г. Павлодар пр. Кутузова, д. 173
Error with г. Тараз мкр. Жансая, ул. Тауке хана, д. 13Б
Error with г. Актау 25 мкр-н, здание 52/2
Error with г. Уральск 4 мкр-н, д. 28
Error with г. Актобе ул. Нокина, д. 16/1
Error with г. Петропавловск ул. Жамбыла, д. 304
Error with г. Актау 6А мкр-н, д. 11/1
Error with г. Алматы ул. Мустай Карим, д. 12Б
Geocoded in 4.647090196609497 sec.
                                    address_geocoded  displayLongitude  \
0  проспект Республики 52/1, 010000 Нұр-Сұлтан, Қ...          71.42355   
1  проспект Республики 84, 010011 Нұр-Сұлтан, Қаз...          71.42169   
2  проспект Нургисы Тлендиева 3, 010003 Нұр-Сұлта...          71.37892   
3     улица Акжол 27/2, 010000 Нұр-Сұлтан, Қазақстан          71.45193   
4  улица Куйши Дина 29, 010008 Нұр-Сұлтан, Қазақстан          71.48314   

   displayLatitude                  address  
0         51.18184  пр. Республики, д. 52/1  
1         51.19071    пр. Ре

In [64]:
df = pd.read_csv("autoservices_geocoded_df.csv",sep=",",encoding='utf-8')

gdf = gpd.GeoDataFrame(df,geometry = gpd.points_from_xy(df.displayLongitude,df.displayLatitude))
gdf.head()
gdf.to_file('autoservices_geocoded.geojson',driver='GeoJSON')